In [6]:
!pip install google-generativeai


In [3]:
!pip install pymupdf

In [7]:
import google.generativeai as genai

## Test Gemini API Key
- gemini-2.0-flash

In [8]:
import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


In [9]:
api_key = os.getenv("GOOGLE_API_KEY")


In [10]:
genai.configure(api_key=api_key)

# Initialize the Gemini model
model = genai.GenerativeModel("gemini-2.0-flash")

# Ask a question
response = model.generate_content("Explain quantum computing in simple terms.")

# Print the response
print(response.text)

Okay, imagine regular computers like light switches: they can be either ON (representing 1) or OFF (representing 0).  That's how they store and process information - bits.

Quantum computers are more like dimmer switches.  Instead of just being ON or OFF, they can be ON, OFF, or *somewhere in between*, or even both at the same time! This "both at the same time" thing is called **superposition**.

Think of it like this:

*   **Classical Bit (like a light switch):**  Either 0 or 1.
*   **Qubit (quantum bit, like a dimmer switch):**  Can be 0, 1, OR a combination of both.  It's in a "probability cloud" of being 0 or 1.

**Why is that useful?**

Because they can explore many possibilities simultaneously.  Imagine trying to find your way out of a maze. A regular computer might try each path one at a time. A quantum computer, because of superposition, could try *all the paths at once* and figure out the solution much faster.

**Another key concept: Entanglement**

Imagine two of our dimmer s

In [11]:
import fitz
import difflib
import re
import json

## Retrieve content for each legal document 

In [12]:
def read_pdf(file_path):
    """Reads a PDF file"""
    doc = fitz.open(file_path)
    return "\n".join([page.get_text() for page in doc])

def read_legal_files(folder_path):
    """Reads all legal files in the folder and returns a dictionary with file names and content."""
    legal_documents = {}

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith(".pdf"):
            legal_documents[file_name] = read_pdf(file_path)
        else:
            print(f"Skipping unsupported file: {file_name}")

    return legal_documents

In [13]:
folder_path = "full_contract_pdf/Part_I/Affiliate_Agreements/"
legal_docs = read_legal_files(folder_path)

# Display first document
for file_name, content in legal_docs.items():
    print(f"--- {file_name} ---\n{content[:500]}...\n")
    break  # Only show the first one for preview

Skipping unsupported file: perturbed_UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf.txt
Skipping unsupported file: .ipynb_checkpoints
--- UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf ---
                                                                   EXHIBIT 10.11
                         NETWORK 1 FINANCIAL CORPORATION
                           AFFILIATE OFFICE AGREEMENT
THIS  AGREEMENT  is  entered  into  by  and  between  NETWORK  1 FINANCIAL, INC.
("NETWORK  1"),  a  Virginia Corporation with its principal place of business at
1501  Farm  Credit  Drive,  Suite 1500, McLean, Virginia 22102-5004, and Payment
Data  Systems,  Inc.,  the  Affiliate Office ("AFFILIATE"), a Nev...



In [14]:
for file_name, content in legal_docs.items():
    print(f"--- {file_name} ---...\n")


--- UsioInc_20040428_SB-2_EX-10.11_1723988_EX-10.11_Affiliate Agreement 2.pdf ---...

--- DigitalCinemaDestinationsCorp_20111220_S-1_EX-10.10_7346719_EX-10.10_Affiliate Agreement.pdf ---...

--- LinkPlusCorp_20050802_8-K_EX-10_3240252_EX-10_Affiliate Agreement.pdf ---...

--- CreditcardscomInc_20070810_S-1_EX-10.33_362297_EX-10.33_Affiliate Agreement.pdf ---...

--- SouthernStarEnergyInc_20051202_SB-2A_EX-9_801890_EX-9_Affiliate Agreement.pdf ---...

--- UnionDentalHoldingsInc_20050204_8-KA_EX-10_3345577_EX-10_Affiliate Agreement.pdf ---...

--- SteelVaultCorp_20081224_10-K_EX-10.16_3074935_EX-10.16_Affiliate Agreement.pdf ---...

--- TubeMediaCorp_20060310_8-K_EX-10.1_513921_EX-10.1_Affiliate Agreement.pdf ---...

--- CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.pdf ---...



In [15]:
def normalize_text(text):
    """
    Normalizes text by removing extra spaces, line breaks, and ensuring consistent spacing.
    """
    text = text.replace("\n", " ")  # Replace newlines with space
    text = re.sub(r"\s+", " ", text).strip()  # Replace multiple spaces with a single space
    return text

## Create and store tagged modified file from its respective json lock file

In [16]:
def apply_perturbation_from_json(original_text, json_file, output_folder="test_benchmark_dataset/ambiguity_inText_contradiction/modified_files_tags/"):
    """
    Reads the JSON metadata and applies the described perturbations to the original document,
    adding unique <*$p$*> markers around the modified sections.

    Parameters:
    - original_text (str): The original contract text.
    - json_file (str): Path to the JSON file containing the perturbation details.
    - output_folder (str): Folder to save the modified contract.

    Returns:
    - modified_text (str): The full modified document.
    """

    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)
    
    print("This is json file:", json_file)
    
    # Load the JSON metadata
    with open(json_file, "r", encoding="utf-8") as file:
        json_data = json.load(file)

    print("file loaded") 
    if isinstance(json_data, list) and len(json_data) > 0:
        json_data = json_data[0]  # Extract the first item in the list
    
    # Normalize the original contract text
    normalized_text = normalize_text(original_text)

    # Apply modifications with unique markers
    modified_text = normalized_text
    
    for perturbation in json_data["perturbation"]:
        original_section = normalize_text(perturbation["original_text"])  
        #print("this is original text:", original_section)
        changed_section = normalize_text(perturbation["changed_text"])
        #print("this is the changed text:", changed_section)
        
        # Wrap changed section with unique <*$p$*> markers
        marked_section = f"<*$p$*>{changed_section}<*$p$*>"

        # Replace original section with marked modified section
        if original_section in modified_text:
            modified_text = modified_text.replace(original_section, marked_section)
        else:
            print(f"Warning: Could not find section in text: {original_section}")
            error = "COULD NOT MODIFY FILE"
            return error

    # Save the modified contract as a new file
    modified_file_name = f"modified_{json_data['file_name']}.txt"
    modified_file_path = os.path.join(output_folder, modified_file_name)

    with open(modified_file_path, "w", encoding="utf-8") as file:
        file.write(modified_text)

    return modified_text

## Prompt to read through legal file and insert different types of perturbations
- 10 different types of prompts to switch
- returns output in json format, which would be considered as lock-file

In [22]:
def generate_perturbation_new(original_text, file_name):    
    """Generates a perturbed version of the legal document section in structured JSON format."""
    
    prompt_ambiguity_inText = f"""You are a contract analyst ensuring consistency in legal agreements. Your task is to modify a document by introducing conflicting definitions of the same term within different sections.

    ### **Definition:**
    Ambiguities also occur when key terms are **inconsistently defined within the document itself**, creating internal contradictions. This type of **in-text contradiction** confuses contract enforcement by allowing multiple interpretations of the same term in different sections, leading to potential legal disputes over meaning.

    ### **Step-by-Step Instructions:**
    1. Identify a **key term** in the contract.
    2. Modify its definition in different sections so that they **conflict**.
    3. Ensure the contradiction creates **uncertainty in enforcement**.
    4. Output the modified contract in structured JSON format.

    ---

    ### **Examples of Conflicting Definitions of a Term:**

    **Example 1:**
    - **Section 2:** "The term ‘Client’ refers to any party who purchases our services."
    - **Section 8:** "Clients are defined as parties who hold an active contract for at least six months."
    - **Explanation:** The first definition includes all buyers, while the second restricts "Clients" to long-term customers, leading to disputes over **contractual obligations**.

    **Example 2:**
    - **Section 1:** "Employees are eligible for health benefits upon hiring."
    - **Section 6:** "Employees are eligible for health benefits after 90 days."
    - **Explanation:** The contradiction makes it unclear when benefits apply.

    **Example 3:**
    - **Section 4:** "The vendor is responsible for product warranties."
    - **Section 9:** "Warranty claims shall be processed by the manufacturer."
    - **Explanation:** It’s unclear **who holds warranty responsibility**.

    **Example 4:**
    - **Section 3:** "Data shall be stored for five years."
    - **Section 12:** "Data must be deleted within two years."
    - **Explanation:** A **conflicting data retention period** could violate regulations.

    **Example 5:**
    - **Section 2:** "A shareholder is anyone holding 5% or more equity."
    - **Section 7:** "A shareholder is anyone with voting rights."
    - **Explanation:** A shareholder under one section may **not qualify in another**.
    
    **Example 6:**
    - **Original:** "The Supplier shall deliver all goods in accordance with the specifications outlined in Schedule A. Any failure to meet these specifications will be considered a material breach of contract. In such cases, the Buyer reserves the right to request a full refund or demand replacement goods within 30 days. The Supplier guarantees that all goods will conform to the quality and performance standards described in the agreement."
    - **Modified:** "The Supplier shall make every effort to deliver goods in accordance with the specifications outlined in Schedule A. If specifications are not fully met, the Buyer and Supplier will engage in discussions to determine an appropriate resolution, which may include partial refunds or adjusted delivery timelines."
    - **Explanation:** This change introduces ambiguity by replacing 'shall deliver' with 'shall make every effort,' making performance non-binding. The original contract clearly stated that failure to meet specifications was a material breach with predefined consequences, whereas the modified version introduces an uncertain resolution process. This creates an **internal contradiction** if another part of the contract enforces strict adherence to Schedule A’s specifications.

    ---

    ### **Return JSON Format**

    {{
        "file_name": {file_name},
        "perturbation": [
            {{
                "type": "Ambiguities - In Text Contradiction",
                "original_text": "EXCERPT BEFORE CHANGE",
                "changed_text": "EXCERPT AFTER CHANGE",
                "explanation": "WHY THIS CHANGE INTRODUCES A PERTURBATION",
                "location": "SECTION OR PARAGRAPH NUMBER"
            }}
        ]
    }}
    
    Below is the original legal text:
    -------------------
    {original_text}
    -------------------

    Now, return ONLY the structured JSON object with the modified text and explanation.
    """
    response = model.generate_content(prompt_ambiguity_inText)
    
    if not response:
        return "ERROR: No response from API"

    return response.text if response else "ERROR: No response from API"

## Functions to clean and apply highlighting to the perturbed legal documents 

In [18]:
def highlight_changes(original, modified):
    """Compares original and modified text and marks changes."""
    original_lines = original.split("\n")
    modified_lines = modified.split("\n")

    diff = difflib.ndiff(original_lines, modified_lines)
    highlighted = []
    
    for line in diff:
        if line.startswith("+ "):  # Added text
            highlighted.append(f"[MODIFIED] {line[2:]}")
        elif line.startswith("- "):  # Removed text
            highlighted.append(f"[REMOVED] {line[2:]}")
        else:
            highlighted.append(line[2:])  
    
    return "\n".join(highlighted)

In [19]:
def extract_clean_text(perturbed_text):
    """
    Removes [MODIFIED], [REMOVED] tags and explanations, leaving only the modified version.
    """
    # Remove any [MODIFIED] or [REMOVED] markers
    clean_text = re.sub(r"\[MODIFIED\]|\[REMOVED\]", "", perturbed_text)
    
    # Remove explanations (assuming they are after a certain marker like "Explanation:")
    clean_text = re.sub(r"Explanation:.*", "", clean_text, flags=re.DOTALL)
    
    # Clean up extra spaces that may remain after removal
    clean_text = re.sub(r"\n\s*\n", "\n", clean_text).strip()
    
    
    
    return clean_text

## Starting of pipeline 
1. reads legal documents
2. calls LLM to add perturbations
3. creates output in json format
4. stores output files in benchmark dataset

In [25]:
def apply_perturbations(folder_path, perturbation_type="contradiction"):
    legal_docs = read_legal_files(folder_path)
    

    for i, (file_name, content) in enumerate(legal_docs.items()):
        results = []
        # if i >= 10:  # Stop after processing 5 documents
        #      break
        print(f"Processing {file_name}...")
        #perturbed_json = generate_perturbation(content, file_name, perturbation_type)
        perturbed_json = generate_perturbation_new(content, file_name)
        #print("This is the perturbed json:", perturbed_json)
        clean_json_text = re.sub(r"```json|```", "", perturbed_json).strip()

        # print('this is json:', clean_json_text)
        try:
            # Convert response into a Python dictionary
            perturbed_data = json.loads(clean_json_text)
            results.append(perturbed_data)
        except json.JSONDecodeError:
            print(f"Error parsing JSON for {file_name}, skipping...")
            continue
            
            
        # Save the modified text
        folder_path = "test_benchmark_dataset/ambiguity_inText_contradiction/"

        # Save the JSON output
        json_output_path = os.path.join(folder_path, f"perturbed_{file_name}.json")
        with open(json_output_path, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        
        json_file_path = f"test_benchmark_dataset/ambiguity_inText_contradiction/perturbed_{file_name}.json"
        modified_contract = apply_perturbation_from_json(content, json_file_path)
        print("________________________________________________________________________")

    print(f"All perturbations saved in {folder_path}")

    return perturbed_json

In [24]:
folder_path = "full_contract_pdf/Part_I/Co_Branding/"
perturbation_type = "contradiction"  # Change to "ambiguity", "omission", etc.
perturbed_legal_docs = apply_perturbations(folder_path, perturbation_type)

Skipping unsupported file: .ipynb_checkpoints
Processing NeoformaInc_19991202_S-1A_EX-10.26_5224521_EX-10.26_Co-Branding Agreement.pdf...
This is json file: test_benchmark_dataset/ambiguity_inText_contradiction/perturbed_NeoformaInc_19991202_S-1A_EX-10.26_5224521_EX-10.26_Co-Branding Agreement.pdf.json
file loaded
________________________________________________________________________
Processing LeadersonlineInc_20000427_S-1A_EX-10.8_4991089_EX-10.8_Co-Branding Agreement.pdf...
This is json file: test_benchmark_dataset/ambiguity_inText_contradiction/perturbed_LeadersonlineInc_20000427_S-1A_EX-10.8_4991089_EX-10.8_Co-Branding Agreement.pdf.json
file loaded
________________________________________________________________________
Processing RaeSystemsInc_20001114_10-Q_EX-10.57_2631790_EX-10.57_Co-Branding Agreement.pdf...
This is json file: test_benchmark_dataset/ambiguity_inText_contradiction/perturbed_RaeSystemsInc_20001114_10-Q_EX-10.57_2631790_EX-10.57_Co-Branding Agreement.pdf.json
